In [1]:
import osmnx as ox
import geopandas as gpd
from fastkml import kml
from shapely.geometry import shape
import pandas as pd
import warnings


c:\Users\khong\Documents\Python Projects\Drive\Drive\venv\Lib\site-packages\fastkml\config.py:39: UserWarning: Package `lxml` missing. Pretty print will be disabled
  warnings.warn("Package `lxml` missing. Pretty print will be disabled")  # noqa: B028


In [2]:
def fetch_roads(place_name, network_type='drive'):
    print(f"Fetching road network for: {place_name}")
    G = ox.graph_from_place(place_name, network_type=network_type)
    nodes, edges = ox.graph_to_gdfs(G, nodes=True, edges=True)
    edges = edges.to_crs("EPSG:4326")
    nodes = nodes.to_crs("EPSG:4326")
    return nodes, edges

def parse_kml_curvature(kml_path):
    print(f"Parsing curvature data from: {kml_path}")
    curvature_gdf = gpd.read_file(kml_path, driver='KML')
    curvature_gdf['curvature'] = [int(float(x[37:42])) for x in curvature_gdf["Description"]]
    return curvature_gdf

def join_curvature(edges, curvature_gdf):
    print("Joining curvature data to edges...")
    edges_with_curviness = gpd.sjoin_nearest(
        edges,
        curvature_gdf[['geometry', 'curvature']],
        how="left",
        max_distance=0.0005,  # ~50 meters
        distance_col="join_dist"
    )
    edges_with_curviness['curvature'] = edges_with_curviness['curvature'].fillna(0)
    edges_with_curviness.drop(columns=['join_dist', 'index_right'], inplace=True)
    return edges_with_curviness

def save_outputs(edges, nodes, save_path, output_prefix="roads_with_curvature"):
    print(f"Saving data to {output_prefix}.csv and .geojson")
    edges.to_csv(f"{save_path}{output_prefix}.csv", index=False)
    edges.to_file(f"{save_path}edges_{output_prefix}.gpkg", layer='edges', driver='GPKG')
    nodes.to_file(f"{save_path}nodes_{output_prefix}.gpkg", layer='nodes', driver='GPKG')



In [5]:

place = "Peak District"  # 🔁 Change this as needed
kml_path = "data/great-britain.c_300.kml"               # 🔁 Replace with your file
save_path = "data/"

nodes, edges = fetch_roads(place)


Fetching road network for: Peak District


In [10]:
curvature_gdf = parse_kml_curvature("C:\\Users\khong\Documents\Python Projects\Drive\Drive\data\great-britain.c_300.kml")


<>:1: SyntaxWarning: invalid escape sequence '\k'
<>:1: SyntaxWarning: invalid escape sequence '\k'
C:\Users\khong\AppData\Local\Temp\ipykernel_14048\1674978136.py:1: SyntaxWarning: invalid escape sequence '\k'
  curvature_gdf = parse_kml_curvature("C:\\Users\khong\Documents\Python Projects\Drive\Drive\data\great-britain.c_300.kml")


Parsing curvature data from: C:\Users\khong\Documents\Python Projects\Drive\Drive\data\great-britain.c_300.kml


In [ ]:
enriched_edges = join_curvature(edges, curvature_gdf)


Joining curvature data to edges...


c:\Users\khong\Documents\Python Projects\Drive\Drive\venv\Lib\site-packages\geopandas\array.py:403: UserWarning: Geometry is in a geographic CRS. Results from 'sjoin_nearest' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  warnings.warn(


In [46]:
print(len(edges))
print(len(enriched_edges))

5367
6033


In [18]:
G = ox.graph_from_gdfs(nodes, edges)
G = ox.graph_from_gdfs(nodes, enriched_edges)

ValueError: `gdf_nodes` and `gdf_edges` must each be uniquely indexed.

In [ ]:

save_outputs(enriched_edges, nodes, save_path)